In [1]:
import pandas as pd
import sys
sys.path.append("creation_functions")
from creation_functions.utilities import create_time_series, preprocessing, scale_readings
from creation_functions.feature_extraction import extract_features

In [2]:
df_train,to_drop = preprocessing(pd.read_csv("windowed_dataset.csv", index_col=0))
X_train = df_train.drop(["class", "subject", "trial"], axis=1)
y_train = df_train["class"]

K_FEATURES = 40

C:\Users\marco\Desktop\università\magistrale\mobile and social sensing systems\ServerPython\ActivityRecognizer\creation_functions\utilities.py:88: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = only_numeric_dataset.corr().abs()


In [3]:
from sklearn.feature_selection import SelectKBest
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import RobustScaler
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

tree_pipe=Pipeline([
    ("scaler", RobustScaler()),
    ("sampling", SMOTE()),
    ("feature-selector", SelectKBest(k=K_FEATURES)),
    ("classifier", RandomForestClassifier())
])
tree_pipe.fit(X_train, y_train)
print("Model Trained")

Model Trained


In [4]:
def compress_df(df):
    # re ordering columns
    df = df[["gravity.x", "gravity.y", "gravity.z", "rotationRate.x", "rotationRate.y",
                         "rotationRate.z", "userAcceleration.x", "userAcceleration.y", "userAcceleration.z"]]
    scaled_dataset = scale_readings(df)
    new_df = extract_features(scaled_dataset, scaled_dataset.shape[0])
    preprocessed_dataset = new_df.drop(to_drop, axis=1)
    return preprocessed_dataset

In [5]:
def recv_msg(sock):
    # Read message length and unpack it into an integer
    msglen = int(sock.recv(8))
    if not msglen:
        return None
    # Read the message data
    return recvall(sock, msglen)

def recvall(sock, n):
    # Helper function to recv n bytes or return None if EOF is hit
    data = bytearray()
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data.extend(packet)
    return data

In [6]:
def msg_to_df(msg):
    samples=msg.split("|")
    if len(samples) == 9:
        # dataset creation
        lin_acc_x=list(map(float, samples[0][1:len(samples[0])-1].split(", ")))
        lin_acc_y=list(map(float, samples[1][1:len(samples[1])-1].split(", ")))
        lin_acc_z=list(map(float, samples[2][1:len(samples[2])-1].split(", ")))
        rot_x=list(map(float, samples[3][1:len(samples[3])-1].split(", ")))
        rot_y=list(map(float, samples[4][1:len(samples[4])-1].split(", ")))
        rot_z=list(map(float, samples[5][1:len(samples[5])-1].split(", ")))
        grav_x=list(map(float, samples[6][1:len(samples[6])-1].split(", ")))
        grav_y=list(map(float, samples[7][1:len(samples[7])-1].split(", ")))
        grav_z=list(map(float, samples[8][1:len(samples[8])-1].split(", ")))
        df_len=min([len(lin_acc_x), len(lin_acc_y), len(lin_acc_z), len(rot_x), len(rot_y), len(rot_z), len(grav_x), len(grav_y), len(grav_z)])

        df=pd.DataFrame(columns=["rotationRate.x", "rotationRate.y", "rotationRate.z", "userAcceleration.x", "userAcceleration.y", "userAcceleration.z", "gravity.x", "gravity.y", "gravity.z"])
        df["rotationRate.x"]=rot_x[0:df_len]
        df["rotationRate.y"]=rot_y[0:df_len]
        df["rotationRate.z"]=rot_z[0:df_len]
        df["userAcceleration.x"]=lin_acc_x[0:df_len]
        df["userAcceleration.y"]=lin_acc_y[0:df_len]
        df["userAcceleration.z"]=lin_acc_z[0:df_len]
        df["gravity.x"]=grav_x[0:df_len]
        df["gravity.y"]=grav_y[0:df_len]
        df["gravity.z"]=grav_z[0:df_len]

        return df

In [7]:
import socket
HOST = ''
PORT = 1060
ADDR =(HOST,PORT)
def server():
    s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    s.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)
    s.bind(ADDR)
    s.listen(1)
    print ('The server is listening at',s.getsockname())
    ss,sockname = s.accept()
    ss,sockname = s.accept()

    while True:
        msg=recv_msg(ss).decode()
        if not msg:
            ss.close()
            return
        phone_df= msg_to_df(msg)
        df_honor_scaled = compress_df(phone_df)
        y_pred = tree_pipe.predict(df_honor_scaled)
        pred = repr(y_pred[0])
        print("Activity recognized: "+pred)
        ss.sendall((pred+ "\r\n").encode())

In [8]:
while True:
    server()
    print("Connection lost...")

The server is listening at ('0.0.0.0', 1060)
Activity recognized: 'sit'
Activity recognized: 'sit'
Activity recognized: 'sit'
Activity recognized: 'sit'
Activity recognized: 'sit'
Activity recognized: 'sit'
Activity recognized: 'sit'
Activity recognized: 'sit'
Activity recognized: 'sit'
Activity recognized: 'sit'
Activity recognized: 'sit'
Activity recognized: 'sit'
Activity recognized: 'sit'
Activity recognized: 'wlk'
Activity recognized: 'wlk'
Activity recognized: 'dws'
Activity recognized: 'wlk'
Activity recognized: 'dws'
Activity recognized: 'jog'
Activity recognized: 'jog'
Activity recognized: 'jog'
Activity recognized: 'jog'
Activity recognized: 'jog'
Activity recognized: 'jog'
Activity recognized: 'jog'


ValueError: invalid literal for int() with base 10: b''